# PlanetaRY spanGLES

This is the initialization file of the `Pryngles` package.

## Useful packages

In [1]:
import unittest
import warnings
import dill
import inspect
from copy import deepcopy
warnings.filterwarnings('ignore')

## Jupyter compatibilty

In [2]:
"""
The purpose of the get_ipython class is to provide some response in the python 
script resulting from the conversion of this notebook.

If you want to add another IPyhton function resulting from a magic command to the class, 
please verify in the resulting python script the corresponding IPython command.

For instance, the magic "%matplotlib nbagg" is converted into:

    get_ipython().magic('matplotlib nbagg',globals())

So, the method "magic" should be add to the get_ipython() class.        
"""
from IPython.display import HTML, Image, display
import IPython.core.autocall as autocall
from IPython import get_ipython
import sys

try:
    cfg=get_ipython().config
except AttributeError:
    def Image(url="",filename="",f=""):
        pass
    class get_ipython(object):
        def run_line_magic(self,*args):
            pass
        def run_cell_magic(self,x,y,z):
            pass
        def magic(self,command,scope=globals()):
            pass

#Magics can only be located from here
%load_ext autoreload
%autoreload 2

## Verbosity

In [3]:
"""Verbosity levels:
SIMPLE: Simple messages.
SYSTEM: System operations.
VERIFY: Message to verify operations
DEEP: Deep debugging messages
ALL: All debugging messages
"""
VERB_NONE=0
VERB_SIMPLE=1
VERB_SYSTEM=2
VERB_VERIFY=3
VERB_DEEP=4
VERB_ALL=100

In [4]:
class Verbose(object):
    """Verbose print in the package
    
    Attributes:
        VERBOSITY: int, default = 0:
            Level of verbosity.
            
    Methods:
        print(level,msg):
            Print a message if level<=VERBOSITY.
    
    Example:
    
        Verbose.print(1,"Hello world") #No print
        
        Verbose.print(0,"Hello world") #Print

        Verbose.VERBOSITY=1
        Verbose.print(1,"Hello world") #Print
        
        Verbose.VERBOSITY=2
        Verbose.print(1,"Hello world") #Print
        
        Verbose.VERBOSITY=2
        Verbose.print(4,"Hello world") #No print
    """
    VERBOSITY=VERB_ALL
    def print(level,*args):
        if level<=Verbose.VERBOSITY:
            print("  "*level+f"VERB{level}::{inspect.stack()[1][3]}::",*args)

verbose=Verbose.print

## PrynglesCommon

Many of the classes in Pryngles inherite methods of this common class

In [5]:
class PrynglesCommon(object):
    
    def save_to(self,filename):
        """Save object to a binary file
        
        Parameters:
            filename: string:
                Name of the file where the object will be stored.
                
            compressed: boolean, default = False:
                If True the file will be stored compressed.
        
        Notes:
            Based on https://betterprogramming.pub/load-fast-load-big-with-compressed-pickles-5f311584507e.
        """
        verbose(VERB_SYSTEM,f"Saving object to {filename}")
        pikd = open(filename,"wb")
        dill.dump(self, pikd)
        pikd.close()
            
    def load_from(self,filename,compressed=False):
        verbose(VERB_SYSTEM,f"Loading object from {filename}")
        pikd = open(filename,"rb")
        data = dill.load(pikd)
        pikd.close()
        verbose(VERB_VERIFY,f"Transferring data to new object")
        self.__dict__=data.__dict__
        return data
    
    def __str__(self):
        #Remove private attributes
        return str({k:v for k,v in self.__dict__.items() if k[0]!='_'})

## Pryngles modules

In [6]:
from pryngles.version import *
from pryngles.util import *
from pryngles.consts import *
from pryngles.science import *
from pryngles.plot import *
from pryngles.sampler import *
from pryngles.spangler import *
from pryngles.body import *
from pryngles.system import *
from pryngles.legacy import *
#Reset verbosity
Verbose.VERBOSITY=VERB_NONE

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Tests

In [7]:
if IN_JUPYTER:
    def test_common(self):
        
        import numpy as np
        import pandas as pd
        import os
        
        Verbose.VERBOSITY=VERB_ALL
        
        p=PrynglesCommon()
        p.casa=dict(perro=0,gato=3)
        p.data=pd.DataFrame(np.random.rand(4000,100))
        p.save_to("/tmp/save.pck")
        print("File size:",os.path.getsize("/tmp/save.pck")/1e6," Mb")
        g=PrynglesCommon()
        g.load_from("/tmp/save.pck")
        print(g.casa,np.array(g.data).shape)
        
        Verbose.VERBOSITY=VERB_NONE
        
    class Test(unittest.TestCase):pass    
    Test.test_common=test_common
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.

    VERB2::save_to:: Saving object to /tmp/save.pck
File size: 3.200736  Mb
    VERB2::load_from:: Loading object from /tmp/save.pck
      VERB3::load_from:: Transferring data to new object
{'perro': 0, 'gato': 3} (4000, 100)



----------------------------------------------------------------------
Ran 1 test in 0.128s

OK


--End--